## __Cross-match and the spectra dawnload with SDSS by Radius Search__ ##

* __only support a *.csv with ra and dec list__  
  The output file are in the fits file format

In [1]:
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

In [2]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Lenovo\Desktop\polluted white dwalf\sdss\selected_data_18.csv') 
df

,postion,ra,dec,Pwd,phot_g_mean_mag_corrected
0,14,359.990611,29.668975,0.991821,16.326300
1,99,359.943752,-1.700525,0.612871,17.981443
2,100,359.939675,-58.213223,0.992817,17.947067
3,111,359.932326,-7.677648,0.999218,16.865816
4,156,359.910483,-50.729074,0.068750,17.926287
...,...,...,...,...,...
23002,1280150,0.071416,55.468659,0.097213,16.525738
23003,1280192,0.047087,-4.054136,0.995734,17.780184
23004,1280200,0.040974,62.806304,0.890490,17.734549
23005,1280216,0.030479,29.949942,0.740718,15.042710


In [3]:
#将数据提取出并列入列表
seriesRa=df.ra
listRa=list(seriesRa.values)
lstRa = [float(i) for i in listRa]
seriesDec=df.dec
listDec=list(seriesDec.values)
lstDec = [float(i) for i in listDec]

#每50个数据为一组进行打包
t=int(len(df)/50)+1
resultra=[]
for i in range(t):
    #lstRa[i*50:(i+1)*50]
    resultra.append(lstRa[i*50:(i+1)*50])
resultra?
resultdec=[]
for i in range(t):
    #lstRa[i*50:(i+1)*50]
    resultdec.append(lstDec[i*50:(i+1)*50])
resultdec?

Type:        list
String form: [[29.668974541706607, -1.7005247249562931, -58.2132226076015, -7.67764761454813, -50.729073884878 <...> 5.46865897340833, -4.054136133682911, 62.806304346375725, 29.94994150605434, -6.135075170529127]]
Length:      461
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.

In [4]:
#准备
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy.table import QTable
import astropy.units as u
import numpy as np
from astropy.table import Table, vstack

In [5]:
from tqdm import tqdm,trange
import time
import threading

In [6]:
#数据匹配
m=[]
x=len(resultdec)
for i in trange(x):
    pos = coords.SkyCoord(resultra[i],resultdec[i], frame='icrs',unit='deg')#输入赤经纬坐标（接下来我们需要考虑如何同时输入多个坐标交叉表、匹配出表格）
    xid = SDSS.query_region(pos, spectro=True)
    #if(xid == None):#匹配结果为零时输出
     #   print("hahah")
      #  continue
    try:
        xid['run2d'] = xid['run2d'].astype('str')#若想将
        m.append(xid)
    except:
        #print(type(xid))
        #print("失败了")
        continue
print(m)

  0%|          | 0/461 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\astroquery\sdss\core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|██████████| 461/461 [00:18<00:00, 25.06it/s]


[<Table length=5>
       ra               dec        ...      specobjid       run2d 
    float64           float64      ...        int64          str7 
---------------- ----------------- ... ------------------- -------
359.713767842835  15.5947199281515 ... 6911917773934272512 v5_10_0
 359.60750442777 -10.5703413133608 ...  731899021953624064      26
 359.60750442777 -10.5703413133608 ... 8069428097231855616 v5_10_0
359.691933535142  27.5015404972475 ... 3155977226755794944      26
359.886650423135 -3.59468005428934 ... 7919751578671620096 v5_10_0, <Table length=3>
       ra              dec        ...      specobjid       run2d 
    float64          float64      ...        int64          str7 
---------------- ---------------- ... ------------------- -------
358.718745980765 6.53472836480932 ... 6927677073625423872 v5_10_0
358.926345821325 35.3595355849103 ... 2116706997876320256      26
358.861828856635 30.9329629329353 ... 7316208205725409280 v5_10_0, <Table length=9>
       ra     

In [7]:
#将各个组匹配出的结果合并在一起判断匹配结果数量（不能直接用于下载光谱：数据量过大且run2d格式已被修改）
t=QTable()
print("len =",len(m))
for i in range(len(m)):
    t=vstack([t,m[i]])
#print(len(t))
print(t)

len = 353
        ra               dec         ...       specobjid         run2d 
----------------- ------------------ ... ---------------------- -------
 359.713767842835   15.5947199281515 ...  6.911917773934273e+18 v5_10_0
  359.60750442777  -10.5703413133608 ...   7.31899021953624e+17      26
  359.60750442777  -10.5703413133608 ...  8.069428097231856e+18 v5_10_0
 359.691933535142   27.5015404972475 ...  3.155977226755795e+18      26
 359.886650423135  -3.59468005428934 ...   7.91975157867162e+18 v5_10_0
 358.718745980765   6.53472836480932 ...  6.927677073625424e+18 v5_10_0
 358.926345821325   35.3595355849103 ... 2.1167069978763203e+18      26
 358.861828856635   30.9329629329353 ...  7.316208205725409e+18 v5_10_0
 357.951445716793   36.3364238544231 ... 2.1167443812716646e+18      26
 358.064353223543 -0.689150732162349 ...  7.701653315104543e+17      26
              ...                ... ...                    ...     ...
 1.26042099948924  -3.16991874324565 ...  4.9146856774

In [8]:
plate=[]
mjd=[]
fiberID=[]
for i in range(len(t)):
    plate.append(t['plate'][i])
    mjd.append(t['mjd'][i])
    fiberID.append(t['fiberID'][i])
#print(plate,mjd,fiberID)

In [9]:
r=int(len(t)/50)+1
p=[]
f=[]
m=[]
for i in range(r):
    p.append(plate[i*50:(i+1)*50])
    m.append(mjd[i*50:(i+1)*50])
    f.append(fiberID[i*50:(i+1)*50])
len(p)

75

In [10]:
#准备
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy.table import QTable
import astropy.units as u
import numpy as np
from astropy.table import Table, vstack

from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

from astropy.io import fits
from concurrent.futures import ThreadPoolExecutor,as_completed
import threading
import os
import time

In [11]:
def Spectra_get(path, plate, mjd, fiberID, data_release = 16):
    for i in range(0, 5):
        time.sleep(0.5)
        try:
            temp = SDSS.get_spectra(plate = int(plate), mjd = int(mjd), fiberID = int(fiberID), data_release = data_release)
            if temp != None:
                temp[0].writeto(path + '/'+ str(plate) + '-' + str(mjd) + '-'+ str(fiberID) + '.fits')
                temp = None
            else:
                print('no data return')
                break
        except:
            if i == 4:
                print('unsuccess to download'+str(plate)+'-'+str(mjd)+'-'+str(fiberID))
        else:
            break

In [12]:
def Multiply_Spectra_get(path, p, m, f, data_release = 16):
    if not os.path.exists(path):
        os.makedirs(path)
    for j in trange(len(p)):
        threads = []
        for i in range(len(p[j])):
            t = threading.Thread(target = Spectra_get, args=(path, p[j][i], m[j][i], f[j][i], data_release))
            threads.append(t)
        for t in (threads):
            t.start()  # 调用start()方法，开始执行
        for t in (threads):
            t.join()

In [13]:
from astropy.io import fits
from concurrent.futures import ThreadPoolExecutor,as_completed
import threading
import os
import time
path =  r'C:\Users\Lenovo\Desktop\polluted white dwalf\2023年秋季学期创新训练计划结题相关材料 (1)\data'
Multiply_Spectra_get(path, p, m, f, data_release = 16)

100%|██████████| 75/75 [03:05<00:00,  2.47s/it]
